# 0- Import Libraries

In [1]:
import pandas as pd
import numpy as np
from StudentsPerformance.utils.common import load_object

# 1- Loading dataset to look at the structure again

In [3]:
dataset = pd.read_csv('..\\data\\01_Raw\\StudentsPerformance.csv')
dataset

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [4]:
dataset.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')

# 2- Defining a function to handle custom data structure

In [5]:
def custom_data(gender: str,
            race_ethnicity: str,
            parental_level_of_education: str,
            lunch: str,
            test_preparation_course: str,
            reading_score: int,
            writing_score: int):
    
    data = pd.DataFrame({
                'gender':[gender],
                'race/ethnicity':[race_ethnicity],
                'parental level of education':[parental_level_of_education],
                'lunch':[lunch],
                'test preparation course':[test_preparation_course],
                'math score':[np.nan],
                'reading score':[reading_score],
                'writing score':[writing_score]
    })

    return data

In [7]:
data_to_predict = custom_data('male',
                              'group A',
                              "bachelor's degree",
                              'standard',
                              'completed',
                              70,
                              74)
data_to_predict

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group A,bachelor's degree,standard,completed,NaN,70,74


# 3- Load Preprocessor and transform custom data

In [8]:
preprocessor = load_object("..\\models\\preprocessor.pkl")
preprocessor

,steps,"[('custom_cleaner', ...), ('outlier_handler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,func,<function cus...00271F7398A40>
,inverse_func,None
,validate,False
,accept_sparse,False
,check_inverse,True
,feature_names_out,None
,kw_args,None


In [9]:
scaled_data = preprocessor.transform(data_to_predict)
scaled_data

2025-09-10 16:19:21,745 - INFO - Applying custom data cleaning rules (no custom rules for now).
2025-09-10 16:19:21,750 - INFO - Checking for boundary outliers in score columns...
2025-09-10 16:19:21,751 - INFO - No boundary outliers found in column 'reading score'.
2025-09-10 16:19:21,753 - INFO - No boundary outliers found in column 'writing score'.
2025-09-10 16:19:21,754 - INFO - No boundary outliers found in the dataset.


array([[0.03079054, 0.36772239, 1.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        ]])

# 4- Load regressor model and predict

In [11]:
regressor = load_object("..\\models\\ridge_model.pkl")
regressor

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'sparse_cg'
,positive,False
,random_state,None


In [12]:
prediction = regressor.predict(scaled_data)[0]

In [13]:
print(f"math score of the data sample is predicted: {prediction}")

math score of the data sample is predicted: 75.0225805425249
